In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString, shape
from PIL import Image
from PIL.ExifTags import TAGS
import os
from shapely.ops import nearest_points

In [ ]:
pd.options.display.max_rows = None

# Define functions

In [2]:
def sum_up_distances(image_index, signal_index, image_geotags):
    if(not (pd.isna(image_index) or pd.isna(signal_index) or np.isnan(image_index) or np.isnan(signal_index))):
        image_index = int(image_index)
        signal_index = int(signal_index)
        distances = image_geotags.distance(image_geotags.shift(-4))
        track_dist = distances[image_index:signal_index]
        track_images = image_geotags[image_index:signal_index]
        lat_diff = track_images.lat.map(float).diff()
        lon_diff = track_images.lon.map(float).diff()
        clean_distances = track_dist[(lon_diff != 0) & (lat_diff != 0)]
        return clean_distances.sum()
    else:
        return None

def get_dist_to_next_signal(df):
    df["signal_idx"] = df.signal_idx.fillna(method="bfill")
    df["increment"] = df.to_crs("EPSG:21781").distance(df.to_crs("EPSG:21781").shift(-1))
    return df.groupby("signal_idx")["increment"].apply(lambda x: x.iloc[::-1].cumsum().iloc[::-1])
    
def get_lat_lon(base_path):
    def f_inner(image_name):
        image = Image.open(base_path + image_name)
        exifdata = image.getexif()
        lat, lon = [exifdata.get(34853)[i] for i in [2,4]]
        return pd.Series({"fn": image_name, "lat": lat, "lon": lon})
    return f_inner

In [13]:
project_dir = "/Users/Georg/Dropbox/Work/projects/2020_09_05_hackathon_zurich"
data_dir = f"{project_dir}/data/Dataset_complete"
image_dir = f"{project_dir}/Trackpictures/nice_weather/nice_weather_filisur_thusis_20200824_pixelated"
results_dir = f"{project_dir}/data/results/"

# Get signal data

In [4]:
trackdata_raw = pd.read_csv(f"{project_dir}/data/Dataset_complete/Trackdata/TrackSiteData_2020_clean.csv")
trackdata = trackdata_raw[[col for col in trackdata_raw.columns if not "Unnamed" in col]]
signals = (trackdata[trackdata["Element Type"].isin(['Distant signal', 'Main & distant signal', 'Main signal'])]
          .dropna(subset=["Latitude", "Longitude"]))
geometry = [Point(xy) for xy in zip(signals.Longitude, signals.Latitude)]
signals_gdf = gpd.GeoDataFrame(signals, geometry=geometry)
signals_gdf.crs = "EPSG:4326"
signals_gdf = signals_gdf.to_crs("EPSG:4326")
(signals_gdf
 .to_csv(f"{project_dir}/data/Dataset_complete/Trackdata/TrackSiteData_2020_clean_geo.csv", index = False))

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (2,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Get railway lines

In [5]:
rail_lines = gpd.read_file(f"{project_dir}/data/railways/railway_lines_wgs84/railway_lines.shp")
rail_lines.to_csv(f"{project_dir}/data/railways/railway_lines.csv")

# Get geotags from images

In [6]:
image_names = pd.Series([fn for fn in os.listdir(directory_name) if "image" in fn])
image_geotags = image_names.apply(get_lat_lon(directory_name))

In [7]:
image_geotags_gdf = gpd.GeoDataFrame(
    image_geotags, geometry=gpd.points_from_xy(image_geotags.lon,image_geotags.lat))
image_geotags_gdf.crs = "EPSG:4326"
image_geotags_gdf = image_geotags_gdf.to_crs("EPSG:4326")
image_geotags_gdf = image_geotags_gdf.sort_values("fn").reset_index(drop=True)
image_geotags_gdf.to_csv(f"{project_dir}/data/geotags/nice_weather_filisur_thusis_20200824_geotags.csv")

# Compute upcoming signal

In [8]:
image_geotags_gdf["signal_idx"] = None
for i in signals_gdf.index:
    nearest_geom = nearest_points(signals_gdf.loc[i]["geometry"], image_geotags_gdf.geometry.unary_union)[1]
    nearest = image_geotags_gdf["geometry"] == nearest_geom
    signal_index = image_geotags_gdf[nearest].index[0]
    image_geotags_gdf.loc[signal_index, "signal_idx"] = i
    
image_geotags_gdf["signal_idx"] = image_geotags_gdf.signal_idx.fillna(method="bfill")
image_geotags_gdf["distance_to_next_signal"] = get_dist_to_next_signal(image_geotags_gdf)

In [14]:
image_geotags_gdf.to_csv(results_dir + "distance_to_next_signal.csv", index = False)

# Investigate EXIF data

In [ ]:
image = Image.open(directory_name + image_name)
exifdata = image.getexif()

In [ ]:
for tag_id in exifdata:
    # get the tag name, instead of human unreadable tag id
    tag = TAGS.get(tag_id, tag_id)
    data = exifdata.get(tag_id)
    # decode bytes 
    if isinstance(data, bytes):
        data = data.decode()
    print(f"{tag:25}: {data}")